In [9]:
import glob
import pandas as pd
import numpy as np
#그래픽
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font_scale = 1.5)
#전처리
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder 
#데이터분리
from sklearn.model_selection import train_test_split
#모델구축
from sklearn.tree import DecisionTreeClassifier
#모델검정
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,make_scorer
from sklearn.metrics import roc_curve
#최적화
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import learning_curve, validation_curve
#하이퍼파라미터 튜닝
from sklearn.model_selection import GridSearchCV


In [10]:
find_file = glob.glob("*Bank.csv*")
find_file

['UniversalBank.csv']

In [13]:

bank_df = pd.read_csv(find_file[0])
bank_df.head()


,ID,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,PersonalLoan,SecuritiesAccount,CDAccount,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
